In [ ]:
import pandas as pd
import numpy as np
import os
from geopy.geocoders import Nominatim
from geopy.point import Point
import warnings
warnings.filterwarnings("ignore")

### Los datos son confidenciales

In [ ]:
# Estableciendo directorio
tar = os.chdir(r"undisclosed")

# Leyendo base
df_contacts = pd.read_csv("undisclosed")

In [ ]:
df_contacts.shape

In [ ]:
df_contacts.sample(n = 2000)

In [ ]:
# Slicing dataframe
df_contacts_1 = df_contacts.iloc[0:499]
df_contacts_2 = df_contacts.iloc[500:999]
df_contacts_3 = df_contacts.iloc[1000:1499]
df_contacts_4 = df_contacts.iloc[1500::]

### Definiendo funciones necesarias

In [ ]:
def reverse_geocoding(lat, lon):
    '''
    Obtiene una string de dirección a partir de coordenadas.
    '''
    geolocator = Nominatim(user_agent="test")
    try:
        location = geolocator.reverse(Point(lat, lon))
        #return location.raw['country']
        #return location.raw['address']['city']
        return location.raw['display_name']
    except:
        return None
    
def retrieve_comuna_chile(row, row_index):
    '''
    Obtiene comuna a partir de una string de dirección.
    '''
    if not row: # check if row is empty
        return None
    try:
        items = row.split(',')
        n = len(items)
        if items[n-1].strip() == 'Chile':
            if items[n-2].strip().isdigit():
                return items[n-5].strip()
            else:
                return items[n-4].strip()
        else:
            return items[n-1].strip()
    except IndexError:
        print(f"IndexError in row {row_index}: {row}. Comuna could not be retrieved.")
        return "could not be retrieved"

### Iterando y obteniendo dirección a través de los data frames

In [ ]:
counter = 0

for df in (df_contacts_1, df_contacts_2, df_contacts_3, df_contacts_4):
    # Contando loop
    counter += 1
    counter_str = str(counter)
    
    # Se limpian las coordenadas
    df['new_latitude'] = df['latitude'].str.replace('\'', '')
    df['new_longitude'] = df['longitude'].str.replace('\'', '')
    
    # Obtener dirección a partir de coordenadas
    df['address'] = np.vectorize(reverse_geocoding)(df['new_latitude'], df['new_longitude'])
    
    # Obtener comuna a partir de dirección
    df['comuna'] = df.apply(lambda row: retrieve_comuna_chile(row['address'], row.name), axis=1)
    
    # Guardando df
    df.to_csv(f'df_contacts_located_{counter_str}.csv')
    df.to_excel(f'df_contacts_located_{counter_str}.xlsx')
    
    # Marcando loop completado
    print(f'El loop número {counter_str} se ha completado')